In [1]:
import glob
import pandas as pd
import numpy as np 
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import torch
from torch import nn
import random

# Personnal Import 
from utilities import DataSet, get_batch,get_mode_date2path
from load_data import load_subway_15_min
from dl_models.graph_conv import graphconv

# Data
- Validation individuelles, aggrégée 3min.
- Metro 15 min (entrée/sortie)

Idée : Identifier des coupure de métro longue :
    - entrée et sortie inhabituelles, montre que le métro est suspendu et laisse les portes ouvertes
    - Sortie plus faible (coupure > 15min)
Evaluer les qualités de prédiction sur ces moments là.

## Data Description
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. 
    - Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.

#### Questionnement 
- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

## Load Validation Individuelles
Load 3 df : df_sub, df_tram, df_bus

In [5]:
folder_path = 'data/'
valid_ind_path = folder_path + 'Sub_Tram_11_2019_03_2020'
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']

subway_paths, tramway_paths, bus_paths = sorted(glob.glob(os.path.join(valid_ind_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_tramway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_bus*.csv")))
mode_month2path = get_mode_date2path([subway_paths,tramway_paths,bus_paths],['sub','tram','bus'])

for name in ['sub','tram','bus']:
    globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])

C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[

## Load Subway 15 min

In [10]:
txt_path = "Métro 15 minutes 2019 2020.txt"

df_metro_funi_2019_2020 = load_subway_15_min(folder_path+txt_path)
df_metro_funi_2019_2020.head()

,datetime,Station,Code ligne,in,out
0,2019-01-01 00:00:00,Ampère Victor Hugo,A,2,4.0
1,2019-01-01 00:15:00,Ampère Victor Hugo,A,3,2.0
2,2019-01-01 00:30:00,Ampère Victor Hugo,A,3,7.0
3,2019-01-01 00:45:00,Ampère Victor Hugo,A,1,9.0
4,2019-01-01 01:00:00,Ampère Victor Hugo,A,0,0.0


# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité. De la même manière que l'historique -7d sert de référence, mais ne témoigne pas d'un lien causal.
- Identifier des moments interessants : anomalies sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.


## Plot Coverage for each station, IN, OUT, and IN+OUT

#### Visualisation des flux IN et OUT entre 6h et 24H. En virant les outliers (0.95) type fête des lumières:

In [ ]:
# Import
from plotting import coverage_day_month

In [ ]:
# Init 
freq  = '15min'
columns = 'hour'
index = 'date'
quantile = 0.95

In [ ]:
# Plotting for each station 
for station in df_metro_funi_2019_2020.Station.unique():
    df_tmps = df_metro_funi_2019_2020[df_metro_funi_2019_2020.Station == station]
    df_tmps = df_tmps[df_tmps.datetime.dt.hour > 5]
    in99,out99 = df_tmps['in'].quantile(quantile),df_tmps['out'].quantile(quantile)
    df_tmps.loc[df_tmps['in']>in99,'in'] = in99
    df_tmps.loc[df_tmps['out']>out99,'out'] = out99
    coverage_day_month(df_tmps, freq = freq,index = index,columns = columns,save = station,folder_save = 'save/profile flux 15min filtred outliers 95/')

#### Visualisation des flux IN et OUT pour chacunes des stations, without filtering 

In [ ]:
for station in df_metro_funi_2019_2020.Station.unique():
    df_tmps = df_metro_funi_2019_2020[df_metro_funi_2019_2020.Station == station]
    coverage_day_month(df_tmps, freq = '60min',index = 'date',columns = 'hour',save = station,folder_save = 'save/profile flux')

In [ ]:
sub_df = sub_nov[sub_nov['lane'] =='A'][['COD_TRG','Flow','VAL_DATE']]
sub_df = sub_df.groupby(['COD_TRG','VAL_DATE']).sum()
sub_df = sub_df.reset_index()
sub_df.VAL_DATE = pd.to_datetime(sub_df.VAL_DATE) 

# Reindex date
start,end = sub_df.VAL_DATE.iloc[0],sub_df.VAL_DATE.iloc[-1]
date_index = pd.date_range(start = start,end = end, freq = '3min')
sub_df = sub_df.pivot(index = 'VAL_DATE',columns = 'COD_TRG',values = 'Flow')
sub_df = sub_df.reindex(date_index).fillna(0)

#Reindex columns :
stations = ['PER','AMP','BEL','COR','HOT','FOC','MAS','CHA','REP','GRA','FLA','CUS','BON','SOI']
sub_df = sub_df[stations]
sub_df.head()

## Feature Vector
A définir pour chacune des stations

In [ ]:
freq  ='3min'
freq_min = int(freq.split('min')[0])

time_step_per_hour = int(60/freq_min) #3min agg

historical_len = 7
Days = 1
Weeks = 1
step_ahead = 10


In [ ]:
Ut = DataSet(sub_df,time_step_per_hour=time_step_per_hour)
norm_Ut = Ut.normalize()    # Normalize before getting the "Feature vector"  (or "Feature Tensor")
(X,Y,dates_verif) = norm_Ut.get_feature_vect(step_ahead,historical_len,Days,Weeks)
print('Feature vector shape: ',X.shape)   # Nb Sample, Nb Nodes, Sequence Length
dates_verif.head()

# Ut.df
# Ut.init_df
# Ut.time_step_per_hour
# Ut.mini
# Ut.maxi
# Ut.mean 
# norm_Ut.df.columns



### Load adjacency Matrix: 

In [ ]:
A_indep = torch.diag(torch.ones(len(Ut.df.columns)))   # Matrice d'adjacence identité, personne n'est connecté avec personne
A_Neighbors = torch.sum(torch.stack([torch.diag(torch.ones(len(Ut.df.columns)-abs(i)),i) for i in [-1,0,1]]),dim =0)   #Une seule ligne de métro, donc tri-diagonale
A_learnable = torch.nn.Parameter(torch.randn(len(Ut.df.columns),len(Ut.df.columns)),requires_grad=True)   #Matrice d'adjacence apprentissable

# Then convert into "Laplacian Matrix", or with "random_walk Matrix", or with another one ...
A_indep = 
A_Neighbors =
A_learnable = 

## Temporal Convolution

## Graph Convolution

In [ ]:
class graphconv(nn.Module):
    def __init__(self,c_in,c_out,graph_conv_act_func,K = 2,enable_bias =True):
        super(graphconv,self).__init__()   # Demande a ce qu'on récupère les méthodes de la classe parent :  'nn.module'
        self.c_in = c_in
        self.c_out = c_out
        self.enable_bias = enable_bias
        self.graph_conv_act_func = graph_conv_act_func
        self.K = K
        if torch.cuda.is_available():
            self.weight = nn.Parameter(torch.cuda.FloatTensor(K,c_in,c_out))   # Initialize with wierd weight like 0e-30 or 1e38. Might not be totally adaptated ...
        else :
            self.weight = nn.Parameter(torch.FloatTensor(K,c_in,c_out))
        self.relu = nn.ReLU()

    def forward(self,x,gcnconv_matrix):
        B, C, L, N = x.shape
        n_mat =  gcnconv_matrix.shape[0]

        x = x.reshape(-1, self.c_in)  #[B, C_in, L, N] -> [BLN, C_in]
        x = torch.einsum('ab, cbd->cad',x,self.weight)   # [BLN,C_in], [K,C_in,C_out] -> [K,BLN,C_out]
        x = x.view(self.K, B*L,N,-1)  #[K,BLN,C_out] ->  [K,BL,N,C_out] 
        x = torch.einsum('ecab,ecbd->ecad',gcnconv_matrix,x)  #[n_adj,BL,N1,N2] ,[K,BL,N2,C_out]  -> [K,BL,N1,C_out] 

        if self.enable_bias:
            x = x + self.bias
        
        x = x.view().permute()
        x = self.relu(x)

        return(x)


In [ ]:
x = torch.randn(2,1,3,4)
A_indep = torch.ones(3,3)

C_in = x.shape[1]
C_out = 8
weight = torch.FloatTensor(C_in,C_out)
weight = torch.Tensor([[1,2,3,4,5,6,7,8]])

adj_matrix =A_indep
# x.shape = [B,C,N2,L]
B,C_in,N2,L = x.shape
# A.shape = [n_adj,N1,N2]
N1,N2 = adj_matrix.shape
#A.shape = [n_adj,B,N1,N2]
stacked_adj = adj_matrix.repeat(1,B,1,1)


embedding = torch.einsum('bcnl,ch -> bhnl',x,weight)

In [ ]:
class MGCN_conv(nn.Module):
    def __init__(self,c_in,c_out,n_adj=2,enable_bias =True):
        super(MGCN_conv,self).__init__()
        self.n_adj = n_adj    
        self.c_in = c_in
        self.c_out = c_out
        self.weight = torch.FloatTensor(n_adj,c_in,c_out)  #[C_in,C_out]   ou pour Einsum: [l,h]
        self.bias = torch.FloatTensor(c_out)
        self.relu = nn.ReLU()

    def forward(self,x,adj_matrix):
        # x.shape = [B,N2,L]
        B,N2,L = x.shape
        # A.shape = [n_adj,N1,N2]
        n_adj,N1,N2 = adj_matrix.shape

        #adj_matrix = torch.unsqueeze(adj_matrix,0) #Créer un nouvel axe 
        stacked_adj = adj_matrix.repeat(B,1,1) # Concat tout les "samples" (B batch, L tmeporal dimension) long du nouvel axe
        stacked_adj = stacked_adj.reshape(-1,N1,N2)  # Stack les n_adj adjacency matrix B*L fois     shape: [n_adj*B, N1,N2] et pour einsum : [b,p,n]

        embedding = torch.einsum('bnl,klh -> bknh',x,self.weight)   # Embedding sur les feature de chaque Noeud
        reshaped_embedding = embedding.reshape(-1,N2,self.c_out)    #Stack la dimension k sur l'axe 0  [n_adj*B,N2,C_out]  et pour einsum : [b,n,h]

        convoluted = torch.einsum('bpn,bnh -> bph',stacked_adj,reshaped_embedding) 
        convoluted = convoluted.reshape(self.n_adj,B,N1,self.c_out)      #Unstack la dimension 0, pour séparer les n_adj matrices d'adjacences

        convoluted = self.relu(convoluted + self.bias)

        return(convoluted,embedding,reshaped_embedding)

# Test : 
N1,N2 = 3,3
adj_matrix = torch.stack([torch.diag(torch.ones(N1)),torch.randn(N1,N2)],dim = 0)  # Matrice d'adjacence identité, personne n'est connecté avec personne
n_adj = adj_matrix.shape[0]

B = 4
L = 6
x = torch.randn(B,N1,L)

c_out = 16
gcn_model = MGCN_conv(L,c_out,n_adj)
AXW,embedding,reshaped_embedding = gcn_model(x,adj_matrix)

print("L'embedding a d'abord été opéré sur la dernière dimension (X*W, temporelle), Puis la convolution (A*(XW)) a sommmé les embedding de chacun des voisins (ou Noeud en lien avec le noeud tagret).")
print(f'X.shape: {x.shape}, AXW.shape:{AXW.shape}')

In [ ]:
embedding.shape,reshaped_embedding.shape

In [ ]:
#Bijecction entre un entier, et son label
def int2lab(int,n_adj=2,B=3):
    n = 1+(int-1)//B 
    b = int-(n-1)*B
    return(f'n{n}_b{b}')

def lab2int(lab,n_adj=2,B=3):
    nb = lab.split('_')
    n,b = int(nb[0][1:]),int(nb[1][1:])
    return((n-1)*B+b)

for k in range(1,15):
    print(int2lab(k),lab2int(int2lab(k)))

## Représentation des numéro des cellules du Tensor
Permet de faire des affichages graphique, et de s'assurer que les ".reshape" font ce que l'on souhaite

In [ ]:
def int2lab(integer, n_adj=2, B=3, C=4, N=5, L=6):
    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    l = 1 + ((integer - 1) % L)
    remaining = (integer - 1) // L
    n = 1 + (remaining % N)
    remaining //= N
    c = 1 + (remaining % C)
    remaining //= C
    b = 1 + (remaining % B)
    remaining //= B
    n_adj = 1 + remaining % n_adj

    return f'adj{n_adj}_b{b}_c{c}_n{n}_l{l}'

def lab2int(label, n_adj=2, B=3, C=4, N=5, L=6):
    split_label = label.split('_')
    n_adj = int(split_label[0][3:])
    b = int(split_label[1][1:])
    c = int(split_label[2][1:])
    n = int(split_label[3][1:])
    l = int(split_label[4][1:])

    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    integer = ((l - 1) + (n - 1) * L + (c - 1) * N * L + (b - 1) * C * N * L + (n_adj - 1) * B * C * N * L) + 1
    return integer


for k in range(1,400):
    print(int2lab(k),lab2int(int2lab(k)))

In [ ]:
label = int2lab(k)
label.split('_')

In [ ]:
class GCN_conv(nn.Module):
    def __init__(self,c_in,c_out,graph_conv_act_func,K=1,enable_bias =True):
        super(GCN_conv,self).__init__()

    
    def forward(self,x,adj_matrix):
        # x.shape = [B,C,N2,L]
        B,C,N2,L = x.shape
        # A.shape = [n_adj,N1,N2]
        n_adj,N1,N2 = adj_matrix.shape
        #A.shape = [n_adj,B,N1,N2]
        adj_matrix = torch.unsqueeze(adj_matrix,0) #Créer un nouvel axe 
        stacked_adj = adj_matrix.repeat(B*L,1,1,1) # Concat tout les "samples" (B batch, L tmeporal dimension) long du nouvel axe
        stacked_adj = stacked_adj.reshape(-1,N1,N2)  # Stack les n_adj adjacency matrix B*L fois

        embedding = torch.einsum('bcnl,ch -> bhnl',x,self.weight)   # Embedding Spatial le long de C_in
        embedding = embedding.permute(0,3,2,1)  # [B,L,N2,C_out]
        embedding = embedding.reshape(-1,N2,C_out)  # [BL,N2,C_out]

        convolution = torch.einsum('kbpn,bhnl -> kbpl',stacked_adj,embedding)









In [ ]:
n_adj_matrix = adj_matrix.repeat(2,1,1)
n_adj,N1,N2 = n_adj_matrix.shape
#A.shape = [n_adj,B,N1,N2]
stacked_adj = n_adj_matrix.repeat(B*L,1,1) # [n_adj*B, N1,N2]  -> Concat tout les "samples" long de l'axe 0
stacked_adj.shape,n_adj_matrix.shape

In [ ]:
x = X.unsqueeze(1) #add the channel dimension (here, only "flow')
print(f'X shape: {x.shape}')
x_b = x[:32]
print(f'x_b shape: {x_b.shape}')

GCN = graphconv(c_in = x.shape[1], c_out = 64, K=2, graph_conv_act_func = 'relu',enable_bias=True)  # K =2 dans MRGNN car considère Pattern et Adj matrix
B, C, N, L = x_b.shape
K = 1
c_out = 64
n_mat =  A_indep.shape[0]

print(f'B,C,N,L : [{B},{C},{N},{L}]')

## GCN détaillé 

In [ ]:
weight = nn.Parameter(torch.FloatTensor(K,C,c_out))
bias = nn.Parameter(torch.FloatTensor(c_out))
print(f'x_b: {x_b.shape}, W: {weight.shape}, Cause there are {K} adjacency matrices (K), {C} C_in and {c_out} C_out')
x_b = x_b.reshape(-1, C)  #[B, C_in, L, N] -> [BLN, C_in]
print(f'reshaped x_b: {x_b.shape}, Cause we have to flatten x_b along the Channel axis, and then pass through a SPATIAL embedding (Linear layer, on C_in of each sample,nodes,historical element)')

# Embedding on C_in:  X*W
embedd_c_in = torch.einsum('ab, cbd->cad',x_b,weight)   # [BLN,C_in], [K,C_in,C_out] -> [K,BLN,C_out]n  Propose K embedding de C_in
print('shape of reshaped_xb after K embedding on C_in: ',embedd_c_in.shape)
reshaped_embedd_c_in = embedd_c_in.view(K, B*L,N,-1)  #[K,BLN,C_out] ->  [K,BL,N,C_out] 
print('Embedded feature vect reshaped: ',reshaped_embedd_c_in.shape, '\n')

# Concat Adj Matrix 
batched_adj_matrix = A_indep.repeat(1,B*L,1,1)
print('Adjacency matrix A_indep: ',A_indep.shape,'batched Multi adj_matrix: ',batched_adj_matrix.shape)

# Convolution A*(XW)
convolutionned = torch.einsum('ecab,ecbd->ecad',batched_adj_matrix,reshaped_embedd_c_in)  #[K,BL,N1,N2] ,[K,BL,N2,C_out]  -> [K,BL,N1,C_out] 
print('Convolution A*(XW): ',convolutionned.shape)

#Add bias: 
convolutionned_n_biased = convolutionned + bias

# Reshape and Permute: 
H =convolutionned_n_biased.view(K,B,L,-1,c_out).permute(0,1,4,3,2)
print(f'Reshaped convolution output: {H.shape}')

In [ ]:
permuted_H = output_H.permute(0, 1, 3, 4, 2)
print(permuted_H.shape)

new_c_in = permuted_H.shape[-1]
first_attn = permuted_H.reshape(K,-1,new_c_in)
print(first_attn.shape)  # [K, B*L*N, C_in']

attn_weight = nn.Linear(new_c_in,1)
softmax = nn.Softmax(-1)  #SoftMax on the last dimension 

#Coefficient d'attention 
attn = attn_weight(first_attn)    # "Embedding" du spatial channel 
attn = attn.permute(1,2,0)  # Permute pour avoir la nombre d'adjacency matrix en dernière dimension
attn = softmax(attn)  # Coefficient d'attention pour chaque Matrices d'adjacence et Embedding spatial assicié  (Ici 1 seule matrice d'adjacence)

In [ ]:
(attn == 1).sum()

In [ ]:
a = torch.randn(2,3,4)
a

In [ ]:
if self.enable_bias:
    x = x + self.bias

x = x.view().permute()
x = self.relu(x)